# 予測
https://qiita.com/yasutaka_ono/items/2e52b6a4ef0666282896
https://qiita.com/ITNewcomer/items/513ef4504038b7f935ae
https://algorithm.joho.info/programming/python/pandas-datareader-nikkei-heikin-kabuka/

# 必要なモジュールのインポート

In [123]:
import pandas as pd
import pandas_profiling as pdp
import pandas_datareader.data as web
import numpy as np
import datetime
import pickle
# scikitlearnのアルゴリズムのロード
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier 

# 変数の定義


In [124]:
pram_predict_day=1                   # 1日後の予測をする
pram_predict_plus_percent= 0.00001             # 40%で上昇率、下降の設定
pram_predict_minus_percent=0.00001            # 40%で上昇率、下降の設定
pd.set_option('display.max_columns', 100)
start = datetime.datetime.now() -datetime.timedelta(days=90)
end = datetime.datetime.now()

# 株価ダウンロード関数定義

In [125]:
def nikkei_download():
    # 日経平均
    df_nikkei=web.DataReader('^N225', 'yahoo',start,end)
    print ("配列長さ:",len(df_nikkei))
    df_nikkei=df_nikkei[['Open','Close']]
    df_nikkei = df_nikkei.reset_index()
    return df_nikkei

def dow_download():
    # ダウ平均
    df_dow = web.DataReader('^DJI', 'yahoo',start,end)
    df_dow['dow_compare']=df_dow['Close']/df_dow['Open']
    df_dow=df_dow[['dow_compare']]
    df_dow = df_dow.reset_index()
    print ("配列長さ:",len(df_dow))
    df_dow['Date']=df_dow['Date']+ datetime.timedelta(days=1)
    return df_dow

In [126]:
df_dow


,Date,dow_compare
0,2020-02-25,0.965893
1,2020-02-26,0.992567
2,2020-02-27,0.971373
3,2020-02-28,1.005482
4,2020-03-02,1.043485
...,...,...
58,2020-05-18,1.022335
59,2020-05-19,0.984920
60,2020-05-20,1.004905
61,2020-05-21,0.996330


In [127]:
df_nikkei=nikkei_download()
df_dow=dow_download()

df_fx=pd.merge(df_nikkei,df_dow,how='inner')
df_fx['Close']=df_fx['Close'].astype('float')
df_fx['Open']=df_fx['Open'].astype('float')

display(df_fx.head(3))
display(df_fx.tail(3))

配列長さ: 59
配列長さ: 63


,Date,Open,Close,dow_compare
0,2020-02-26,22374.140625,22426.189453,0.965893
1,2020-02-27,22255.830078,21948.230469,0.992567
2,2020-02-28,21518.009766,21142.960938,0.971373


,Date,Open,Close,dow_compare
44,2020-05-20,20454.490234,20595.150391,0.984920
45,2020-05-21,20692.589844,20552.310547,1.004905
46,2020-05-22,20583.949219,20388.160156,0.996330


# 今朝の日経平均のオープン価格

In [128]:
s=pd.Series(["2020-05-23 00:00:00",20583,0,1.04],index=df_fx.columns)
df_fx=df_fx.append(s,ignore_index=True)

# 関数定義

In [129]:
# 偏差値
def Deviation_value(df_tmp,x):
    std=df_tmp.std()['Open']
    mean=df_tmp.mean()['Open']
    deviation=(mean-x)*10/std+50
    return deviation

# 前処理モジュールの定義（当日の上昇率、過去からの上昇率、下降率）
def result(day0,xday):          #day0 当日の上昇率(close/open)の結果を返す
    rate=0  #上昇率
    if ((i-xday)>=0 and data_len>(i-xday)): 
        rate= ((df_fx.iloc[i,close_index]/df_fx.iloc[i,open_index])-1)*100
    return rate

def persent(day0,xday):         #　当日のopen価格と、過去のと比較して上昇率(当日open/過去open)を返す
    rate=0  #上昇率
    if ((i-xday)>=0 and data_len>(i-xday)):        
        rate= ((df_fx.iloc[i,open_index]/df_fx.iloc[i-xday,open_index])-1)*100
        if rate <0:
            rate=0
        #rate=np.log10(rate)
    return rate

def persent_minus(day0,xday):   #　当日のopen価格と、過去のと比較して下降率(-当日open/過去open)を返す
    rate=0  #上昇率
    if ((i-xday)>=0 and data_len>(i-xday)):        
        rate= ((df_fx.iloc[i,open_index]/df_fx.iloc[i-xday,open_index])-1)*100
        if rate>0:
            rate=0
        #rate=np.log10(rate)
    return rate

In [130]:
df=pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
df = pd.DataFrame(index=['Date'], columns=[
"Date" ,"Close","Open",
"day+1" ,"day+2" ,"day+3",
"day+4" ,"day+5" ,"day+6",
"day+7" ,"day+8" ,"day+9",
"day+10","day+11","day+12",
"day+13","day+14","day+15",
"day+16","day+17","day+18",
"day+19","day+20","day+21",
"day+22","day+23","day+24",
"day+25","day+26","day+27",
"day+28","day+29","day+30",
"day-1" ,"day-2" ,"day-3",
"day-4" ,"day-5" ,"day-6",
"day-7" ,"day-8" ,"day-9",
"day-10","day-11","day-12",
"day-13","day-14","day-15",
"day-16","day-17","day-18",
"day-19","day-20","day-21",
"day-22","day-23","day-24",
"day-25","day-26","day-27",
"day-28","day-29","day-30",
"std-30","std-60","std-90",
"dow_compare",
"label_up","label_none","label_down",
])

In [131]:
df_fx = df_fx.reset_index(drop=True)
data_len=len(df_fx)
date_index=  df_fx.columns.get_loc('Date')
close_index= df_fx.columns.get_loc('Close')
open_index=  df_fx.columns.get_loc('Open')
dow_compare_index=df_fx.columns.get_loc('dow_compare')

In [132]:
# 前処理部分　（現在の値と、過去のからの上昇率、下降率を記したDFを作成
for i in range(data_len) :
    Date_day=df_fx.iloc[i,date_index ]

    if i%5==0:
        print('{0:.1%}'.format(i/data_len),end=" ")

    #当日のOPENの株価
    day0         =df_fx.iloc[i,open_index]
    day0_open    =df_fx.iloc[i,open_index] 
    day0_close   =df_fx.iloc[i,close_index] 
    day0_dow_compare  =df_fx.iloc[i,dow_compare_index] 

    #その後の結果から正解ラベルを作成
    label_up=0     #上昇
    label_none=0   #変化少ない
    label_down=0   #下降

    #正解データの計算  (pram_predict_day) 日後に、(pram_predict_persent)％の前後がある
    prediction=result(day0_open,pram_predict_day)    

    # ３つのらべるの場合
    if prediction>=      pram_predict_plus_percent:
        label_up  =1        #上昇
        label_none=0        #そのまま
        label_down=0        #下降
    elif prediction<=   -pram_predict_minus_percent:
        label_up  =0        #上昇
        label_none=0        #そのまま
        label_down=1        #下降
    else:
        
        label_up  =0        #上昇
        label_none=1        #そのまま
        label_down=0        #下降
    
    #display(df.tail(30))
    Deviation30=Deviation_value(df.tail(30),day0_open)
    Deviation60=Deviation_value(df.tail(60),day0_open)
    Deviation90=Deviation_value(df.tail(90),day0_open)
    
    series=pd.Series([Date_day,day0_close,day0_open,
                      persent(day0,1),persent(day0,2),persent(day0,3),
                      persent(day0,4),persent(day0,5),persent(day0,6),
                      persent(day0,7),persent(day0,8),persent(day0,9),
                      persent(day0,10),persent(day0,11),persent(day0,12),
                      persent(day0,13),persent(day0,14),persent(day0,15),
                      persent(day0,16),persent(day0,17),persent(day0,18),
                      persent(day0,19),persent(day0,20),persent(day0,21),
                      persent(day0,22),persent(day0,23),persent(day0,24),
                      persent(day0,25),persent(day0,26),persent(day0,27),
                      persent(day0,28),persent(day0,29),persent(day0,30),

                      persent_minus(day0,1),persent_minus(day0,2),persent_minus(day0,3),
                      persent_minus(day0,4),persent_minus(day0,5),persent_minus(day0,6),
                      persent_minus(day0,7),persent_minus(day0,8),persent_minus(day0,9),
                      persent_minus(day0,10),persent_minus(day0,11),persent_minus(day0,12),
                      persent_minus(day0,13),persent_minus(day0,14),persent_minus(day0,15),
                      persent_minus(day0,16),persent_minus(day0,17),persent_minus(day0,18),
                      persent_minus(day0,19),persent_minus(day0,20),persent_minus(day0,21),
                      persent_minus(day0,22),persent_minus(day0,23),persent_minus(day0,24),
                      persent_minus(day0,25),persent_minus(day0,26),persent_minus(day0,27),
                      persent_minus(day0,28),persent_minus(day0,29),persent_minus(day0,30),
                      
                      Deviation30,Deviation60,Deviation90,
                      day0_dow_compare,
                      label_up,label_none,label_down,
                     ]
                      ,index=df.columns)
    series.append
    df=df.append(series,ignore_index = True)
# 曜日の追加
df['Date']=pd.to_datetime(df['Date'])
df['weekday']=df['Date'].apply(lambda x: x.weekday())
df=pd.get_dummies(df,columns=['weekday'])

#df['rolling_up5']=df["label_up"].rolling(window=5).mean()
#df['rolling_down5']=df["label_down"].rolling(window=5).mean()
#df['label_none5']=df["label_none"].rolling(window=5).mean()

#df['rolling_up10']=df["label_up"].rolling(window=10).mean()
#df['rolling_down10']=df["label_down"].rolling(window=10).mean()
#df['label_none10']=df["label_none"].rolling(window=10).mean()
df['result']=df['Close']/df['Open']

col = df.columns.tolist() # 列名のリスト
col.remove('label_up')
col.remove('label_none')
col.remove('label_down')
col.remove('result')

col.append('label_up')
col.append('label_none')
col.append('label_down') # 末尾に`t`を追加
col.append('result') # 末尾に`t`を追加

df = df.ix[:,col]


display(df.head(3))
display(df.tail(3))

0.0% 10.4% 20.8% 31.2% 41.7% 52.1% 62.5% 72.9% 83.3% 93.8% 

/Users/fujiccofujimoto/.pyenv/versions/Anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:97: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
/Users/fujiccofujimoto/.pyenv/versions/Anaconda3-2019.10/lib/python3.7/site-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


,Date,Close,Open,day+1,day+2,day+3,day+4,day+5,day+6,day+7,day+8,day+9,day+10,day+11,day+12,day+13,day+14,day+15,day+16,day+17,day+18,day+19,day+20,day+21,day+22,day+23,day+24,day+25,day+26,day+27,day+28,day+29,day+30,day-1,day-2,day-3,day-4,day-5,day-6,day-7,day-8,day-9,day-10,day-11,day-12,day-13,day-14,day-15,day-16,day-17,day-18,day-19,day-20,day-21,day-22,day-23,day-24,day-25,day-26,day-27,day-28,day-29,day-30,std-30,std-60,std-90,dow_compare,weekday_1.0,weekday_2.0,weekday_3.0,weekday_4.0,weekday_5.0,label_up,label_none,label_down,result
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN
1,2020-02-26,22426.189453,22374.140625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.965893,0,1,0,0,0,0,1,0,1.002326
2,2020-02-27,21948.230469,22255.830078,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.528783,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.992567,0,0,1,0,0,0,0,1,0.986179


,Date,Close,Open,day+1,day+2,day+3,day+4,day+5,day+6,day+7,day+8,day+9,day+10,day+11,day+12,day+13,day+14,day+15,day+16,day+17,day+18,day+19,day+20,day+21,day+22,day+23,day+24,day+25,day+26,day+27,day+28,day+29,day+30,day-1,day-2,day-3,day-4,day-5,day-6,day-7,day-8,day-9,day-10,day-11,day-12,day-13,day-14,day-15,day-16,day-17,day-18,day-19,day-20,day-21,day-22,day-23,day-24,day-25,day-26,day-27,day-28,day-29,day-30,std-30,std-60,std-90,dow_compare,weekday_1.0,weekday_2.0,weekday_3.0,weekday_4.0,weekday_5.0,label_up,label_none,label_down,result
46,2020-05-21,20552.310547,20692.589844,1.16405,1.08977,2.69383,2.74124,2.73905,1.36852,3.60753,6.28743,3.5045,2.91913,4.63391,7.0388,7.14311,8.28612,6.22572,5.70468,7.15275,5.63237,8.0536,6.11546,6.79495,8.63529,9.6072,15.2698,15.3792,10.7378,7.8756,8.78258,7.57909,12.1744,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31.106568,40.911111,40.911111,1.004905,0,0,1,0,0,0,0,1,0.993221
47,2020-05-22,20388.160156,20583.949219,0,0.632912,0.559025,2.15466,2.20183,2.19965,0.836314,3.06357,5.7294,2.96108,2.37878,4.08456,6.47682,6.58058,7.7176,5.66801,5.14971,6.59017,5.07778,7.4863,5.55833,6.23425,8.06494,9.03174,14.6646,14.7734,10.1564,7.30923,8.21144,7.01428,-0.525022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34.130291,41.941236,41.941236,0.996330,0,0,0,1,0,0,0,1,0.990488
48,2020-05-23,0.000000,20583.000000,0,0,0.628272,0.554388,2.14995,2.19712,2.19493,0.831664,3.05882,5.72453,2.95633,2.37406,4.07976,6.47191,6.57567,7.71263,5.66314,5.14486,6.58526,5.07294,7.48134,5.55347,6.22936,8.05995,9.02671,14.6593,14.7681,10.1513,7.30428,8.20645,-0.00461145,-0.529609,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,35.360043,42.088743,42.088743,1.040000,0,0,0,0,1,0,0,1,0.000000


# データ整形

In [133]:
testX=df.iloc[:,3:y_loc_s-1]
display(testX)
testX.shape
testX.columns

,day+1,day+2,day+3,day+4,day+5,day+6,day+7,day+8,day+9,day+10,day+11,day+12,day+13,day+14,day+15,day+16,day+17,day+18,day+19,day+20,day+21,day+22,day+23,day+24,day+25,day+26,day+27,day+28,day+29,day+30,day-1,day-2,day-3,day-4,day-5,day-6,day-7,day-8,day-9,day-10,day-11,day-12,day-13,day-14,day-15,day-16,day-17,day-18,day-19,day-20,day-21,day-22,day-23,day-24,day-25,day-26,day-27,day-28,day-29,day-30,std-30,std-60,std-90,dow_compare,weekday_1.0,weekday_2.0,weekday_3.0,weekday_4.0,weekday_5.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.965893,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.528783,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.992567,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-3.31518,-3.82643,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,145.265698,145.265698,145.265698,0.971373,0,0,0,1,0
4,0.622643,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2.71318,-3.22761,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,58.564746,58.564746,58.564746,1.043485,1,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-3.48601,-2.88507,-6.10461,-6.60111,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,74.613644,74.613644,74.613644,0.968424,0,1,0,0,0
6,2.40544,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1.16442,-0.549031,-3.84601,-4.35445,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55.668175,55.668175,55.668175,1.026804,0,0,1,0,0
7,0,0.538836,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1.82276,0,-2.96596,-2.36178,-5.59866,-6.09784,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,62.160378,62.160378,62.160378,0.979355,0,0,0,1,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-7.30569,-8.99528,-6.80622,-10.055,-9.49493,-12.4953,-12.958,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,87.332603,87.332603,87.332603,0.954332,1,0,0,0,0
9,1.45505,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-5.95694,-7.67112,-5.4502,-8.74622,-8.17803,-11.2221,-11.6915,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,67.154216,67.154216,67.154216,1.023112,0,1,0,0,0


Index(['day+1', 'day+2', 'day+3', 'day+4', 'day+5', 'day+6', 'day+7', 'day+8',
       'day+9', 'day+10', 'day+11', 'day+12', 'day+13', 'day+14', 'day+15',
       'day+16', 'day+17', 'day+18', 'day+19', 'day+20', 'day+21', 'day+22',
       'day+23', 'day+24', 'day+25', 'day+26', 'day+27', 'day+28', 'day+29',
       'day+30', 'day-1', 'day-2', 'day-3', 'day-4', 'day-5', 'day-6', 'day-7',
       'day-8', 'day-9', 'day-10', 'day-11', 'day-12', 'day-13', 'day-14',
       'day-15', 'day-16', 'day-17', 'day-18', 'day-19', 'day-20', 'day-21',
       'day-22', 'day-23', 'day-24', 'day-25', 'day-26', 'day-27', 'day-28',
       'day-29', 'day-30', 'std-30', 'std-60', 'std-90', 'dow_compare',
       'weekday_1.0', 'weekday_2.0', 'weekday_3.0', 'weekday_4.0',
       'weekday_5.0'],
      dtype='object')

In [134]:
y_loc_s=df.columns.get_loc('label_up')
y_loc_e=df.columns.get_loc('label_down')

#トレーニングデータと正解ラベルに分割
# 説明変数
df=df.iloc[30:,:]
testX=df.iloc[:,3:y_loc_s-1]

#Numpy形式に変換
testX=testX.as_matrix()
testX.shape

/Users/fujiccofujimoto/.pyenv/versions/Anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


(19, 68)

# sicikit-learn準備

In [135]:
# 保存したモデルをロードする
filename = 'model/sk-learn_model.sav'
classifier = pickle.load(open(filename, 'rb'))

# 予測

In [136]:
df['predict']=np.argmax(classifier.predict_proba(testX),axis=1)
df['predict']=df['predict'].apply(lambda x: "買" if x==0  else "売")
df['result2']=((df['result']-1)*100).round(2)
def check_asnswer(row):
    if row['predict']=="売" and row['result2']<=0:
        return "○"
    if row['predict']=="売" and row['result2']>0:
        return "×"
    if row['predict']=="買" and row['result2']>=0:
        return "○"
    if row['predict']=="買" and row['result2']<0:
        return "×"
    return "-"
df['正誤'] = df.apply(check_asnswer, axis=1)

In [137]:
display(df[['Date','Open','Close','result2','predict','正誤']])
display(df['正誤'].value_counts())

,Date,Open,Close,result2,predict,正誤
30,2020-04-17,19575.849609,19897.259766,1.64,買,○
31,2020-04-21,19479.830078,19280.779297,-1.02,売,○
32,2020-04-22,19109.179688,19137.949219,0.15,買,○
33,2020-04-23,19313.039062,19429.439453,0.60,買,○
34,2020-04-24,19331.859375,19262.000000,-0.36,買,×
35,2020-04-28,19776.179688,19771.189453,-0.03,買,×
36,2020-04-30,20105.679688,20193.689453,0.44,売,×
37,2020-05-01,19991.970703,19619.349609,-1.86,買,×
38,2020-05-07,19468.519531,19674.769531,1.06,買,○
39,2020-05-08,19972.089844,20179.089844,1.04,売,×


×    10
○     9
Name: 正誤, dtype: int64